In [1]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise
import scipy.sparse as sp
from itertools import islice, cycle
#from tqdm.auto import tqdm

# Данные

Изучаем данные

In [2]:
df = pd.read_csv('data/interactions.csv')
df_users = pd.read_csv('data/users.csv')
df_items = pd.read_csv('data/items.csv')

In [4]:
df_users.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [5]:
df_items.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [3]:
df.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [6]:
df['start_date'] = pd.to_datetime(df['start_date'])

In [7]:
duplicates = df.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = df[duplicates].sort_values(by=['user_id', 'start_date'])
df = df[~duplicates]

In [8]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
df = df.append(df_duplicates.reset_index(), ignore_index=True)

In [9]:
df['progress'] = df['progress'].astype(np.int8)
df['rating'] = df['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [10]:
df.to_pickle('data/interactions_preprocessed.pickle')

In [11]:
df_users['age'] = df_users['age'].astype('category')
df_users['sex'] = df_users['sex'].astype(pd.SparseDtype(np.float32, np.nan))

In [12]:
interaction_users = df['user_id'].unique()

common_users = len(np.intersect1d(interaction_users, df_users['user_id']))
users_only_in_interaction = len(np.setdiff1d(interaction_users, df_users['user_id']))
users_only_features = len(np.setdiff1d(df_users['user_id'], interaction_users))
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')


Кол-во пользователей - 158811
Кол-во пользователей c взаимодействиями и фичами - 135677 (85.43%)
Кол-во пользователей только c взаимодействиями - 15923 (10.03%)
Кол-во пользователей только c фичами - 7211 (4.54%)


In [13]:
df_users.to_pickle('data/users_preprocessed.pickle')

In [14]:
for col in ['genres', 'authors', 'year']:
    df_items[col] = df_items[col].astype('category')

In [15]:
interaction_items = df['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, df_items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, df_items['id']))
items_only_features = len(np.setdiff1d(df_items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 59599
Кол-во книг c взаимодействиями и фичами - 59599 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [17]:
df_items.to_pickle('data/items_preprocessed.pickle')

# Метрики

In [19]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


Precision@k

Вначале посчитаем метрик для топ-2 (т.е. К = 2). Алгоритм следующий:

- Релевантные объекты, которые не были рекомендованы игнорируем (NaN)
- Определяем, какие релеватные рекомендации попали в топ-2 (hit)
- True positive для каждого пользователя
Делим TP на K
- Считаем Precision@K для каждого пользователя как сумму его TP/K
- Все Precision@K усредняем

In [20]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Искра жизни', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Искра жизни,2
8,Вася,Зеленая миля,3


In [21]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Зеленая миля                          3.0
        Рита Хейуорт и спасение из Шоушенка   NaN

In [22]:
df_merged['hit@2'] = df_merged['rank'] <= 2
df_merged

rank  hit@2
user_id item_id                                         
Аня     Мастер и Маргарита                    NaN  False
Боря    451° по Фаренгейту                    1.0   True
Вася    Зеленая миля                          3.0  False
        Рита Хейуорт и спасение из Шоушенка   NaN  False

In [23]:
df_merged['hit@2/2'] = df_merged['hit@2'] / 2
df_merged

rank  hit@2  hit@2/2
user_id item_id                                                  
Аня     Мастер и Маргарита                    NaN  False      0.0
Боря    451° по Фаренгейту                    1.0   True      0.5
Вася    Зеленая миля                          3.0  False      0.0
        Рита Хейуорт и спасение из Шоушенка   NaN  False      0.0

In [24]:
df_prec2 = df_merged.groupby(level=0)['hit@2/2'].sum()
df_prec2

user_id
Аня     0.0
Боря    0.5
Вася    0.0
Name: hit@2/2, dtype: float64

In [26]:


users_count = df_merged.index.get_level_values('user_id').nunique()
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    df_merged[hit_k] = df_merged['rank'] <= k
    print(f'Precision@{k} = {(df_merged[hit_k] / k).sum() / users_count:.4f}')

Precision@1 = 0.3333
Precision@2 = 0.1667
Precision@3 = 0.2222


Recall@k

Посчитайте метрику полноты для k = 1, 2, 3

Recall@K (k =2) 

In [ ]:
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

In [ ]:
for k in [1, 2, 3]:
    hit_k = f'hit@{k}'
    # Уже посчитано
    # df_merged[hit_k] = df_merged['rank'] <= k  
    print(f"Recall@{k} = {(df_merged[hit_k] / df_merged['users_item_count']).sum() / users_count:.4f}")

Почему Precision@k и Recall@k не всегда хорошо считать?

MRR

Эта метрика оценивает качество топ-N рекомендаций c учетом рангов/позиций. Основная идея - оценить "попадания" с весом, зависящим от позиции (обычно это обратная пропорциальная зависимость, то есть чем больше позиция, тем меньше вес).

**Mean Reciproal Rank, MRR**


$MRR \equiv \frac{1}{Q} \sum_{i=1}^Q\frac{1}{r_i}$

Где Q - это query или наш пользователь, а rank_i - позиция первой релевантной рекомендации

In [27]:
df_true = pd.DataFrame({
    'user_id': ['Аня',                'Боря',               'Вася',         'Вася'],
    'item_id': ['Мастер и Маргарита', '451° по Фаренгейту', 'Зеленая миля', 'Рита Хейуорт и спасение из Шоушенка'],
})
df_true

,user_id,item_id
0,Аня,Мастер и Маргарита
1,Боря,451° по Фаренгейту
2,Вася,Зеленая миля
3,Вася,Рита Хейуорт и спасение из Шоушенка


In [29]:
df_recs = pd.DataFrame({
    'user_id': [
        'Аня', 'Аня', 'Аня', 
        'Боря', 'Боря', 'Боря', 
        'Вася', 'Вася', 'Вася',
    ],
    'item_id': [
        'Отверженные', 'Двенадцать стульев', 'Герои нашего времени', 
        '451° по Фаренгейту', '1984', 'О дивный новый мир',
        'Десять негритят', 'Рита Хейуорт и спасение из Шоушенка', 'Зеленая миля', 
    ],
    'rank': [
        1, 2, 3,
        1, 2, 3,
        1, 2, 3,
    ]
})
df_recs

,user_id,item_id,rank
0,Аня,Отверженные,1
1,Аня,Двенадцать стульев,2
2,Аня,Герои нашего времени,3
3,Боря,451° по Фаренгейту,1
4,Боря,1984,2
5,Боря,О дивный новый мир,3
6,Вася,Десять негритят,1
7,Вася,Рита Хейуорт и спасение из Шоушенка,2
8,Вася,Зеленая миля,3


In [30]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged = df_merged.sort_values(by=['user_id', 'rank'])
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Рита Хейуорт и спасение из Шоушенка   2.0
        Зеленая миля                          3.0

In [31]:
df_merged['reciprocal_rank'] = 1 / df_merged['rank']
df_merged

rank  reciprocal_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN              NaN
Боря    451° по Фаренгейту                    1.0         1.000000
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000
        Зеленая миля                          3.0         0.333333

In [32]:
mrr = df_merged.groupby(level='user_id')['reciprocal_rank'].max()
mrr

user_id
Аня     NaN
Боря    1.0
Вася    0.5
Name: reciprocal_rank, dtype: float64

In [33]:
print(f"MRR = {mrr.fillna(0).mean()}")

MRR = 0.5


MAP

Написать код подсчета метрики MAP

In [39]:
df_merged = df_true.set_index(['user_id', 'item_id']).join(df_recs.set_index(['user_id', 'item_id']), how='left')
df_merged = df_merged.sort_values(by=['user_id', 'rank'])
df_merged

rank
user_id item_id                                  
Аня     Мастер и Маргарита                    NaN
Боря    451° по Фаренгейту                    1.0
Вася    Рита Хейуорт и спасение из Шоушенка   2.0
        Зеленая миля                          3.0

In [40]:
df_merged['cumulative_rank'] = df_merged.groupby(level='user_id').cumcount() + 1

In [41]:
df_merged

rank  cumulative_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN                1
Боря    451° по Фаренгейту                    1.0                1
Вася    Рита Хейуорт и спасение из Шоушенка   2.0                1
        Зеленая миля                          3.0                2

In [42]:
df_merged['cumulative_rank'] = df_merged['cumulative_rank'] / df_merged['rank']

In [43]:
df_merged

rank  cumulative_rank
user_id item_id                                                   
Аня     Мастер и Маргарита                    NaN              NaN
Боря    451° по Фаренгейту                    1.0         1.000000
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000
        Зеленая миля                          3.0         0.666667

In [44]:
df_merged['users_item_count'] = df_merged.groupby(level='user_id')['rank'].transform(np.size)
df_merged

rank  cumulative_rank  \
user_id item_id                                                      
Аня     Мастер и Маргарита                    NaN              NaN   
Боря    451° по Фаренгейту                    1.0         1.000000   
Вася    Рита Хейуорт и спасение из Шоушенка   2.0         0.500000   
        Зеленая миля                          3.0         0.666667   

                                             users_item_count  
user_id item_id                                                
Аня     Мастер и Маргарита                                1.0  
Боря    451° по Фаренгейту                                1.0  
Вася    Рита Хейуорт и спасение из Шоушенка               2.0  
        Зеленая миля                                      2.0

In [45]:
users_count = df_merged.index.get_level_values('user_id').nunique()

In [46]:
users_count

3

In [47]:
map3 = (df_merged["cumulative_rank"] / df_merged["users_item_count"]).sum() / users_count
print(f"MAP@3 = {map3}")

MAP@3 = 0.5277777777777778


Посчитаем все на нашем датасете

In [49]:
test_dates = df['start_date'].unique()[-7:]
test_dates = list(pairwise(test_dates))
test_dates

[(numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (numpy.datetime64('2019-12-26T00:00:00.000000000'),
  numpy.datetime64('2019-12-27T00:00:00.000000000')),
 (numpy.datetime64('2019-12-27T00:00:00.000000000'),
  numpy.datetime64('2019-12-28T00:00:00.000000000')),
 (numpy.datetime64('2019-12-28T00:00:00.000000000'),
  numpy.datetime64('2019-12-29T00:00:00.000000000')),
 (numpy.datetime64('2019-12-29T00:00:00.000000000'),
  numpy.datetime64('2019-12-30T00:00:00.000000000')),
 (numpy.datetime64('2019-12-30T00:00:00.000000000'),
  numpy.datetime64('2019-12-31T00:00:00.000000000'))]

In [50]:
split_dates = test_dates[0]
train = df[df['start_date'] < split_dates[0]]
test = df[(df['start_date'] >= split_dates[0]) & (df['start_date'] < split_dates[1])]
test = test[(test['rating'] >= 4) | (test['rating'].isnull())]
split_dates, train.shape, test.shape

((numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (1517994, 5),
 (2114, 5))

In [51]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [52]:
pop_model = PopularRecommender(days=7, dt_column='start_date')
pop_model.fit(train)

In [53]:
top10_recs = pop_model.recommend()

In [54]:
item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()
item_titles[128115]

'Ворон-челобитчик'

In [55]:
list(map(item_titles.get, top10_recs))

['Пикник на обочине',
 'Кавказский пленник',
 'Записки юного врача',
 'Богатый папа, бедный папа. Роберт Кийосаки (обзор)',
 'Москва и москвичи',
 'Понедельник начинается в субботу',
 'Хитрость',
 'Сказка о попе и о работнике его Балде',
 'Лорд, который влюбился. Тайный жених',
 'История государства Российского. Том 2. От Великого князя Святополка до Великого князя Мстислава Изяславовича']

In [56]:
recs = pd.DataFrame({'user_id': test['user_id'].unique()})
top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], N=top_N)
recs.head()

,user_id,item_id
0,38753,"[235407, 230067, 35265, 281005, 147734, 208935..."
1,101642,"[235407, 230067, 35265, 281005, 147734, 208935..."
2,13548,"[235407, 230067, 35265, 281005, 147734, 208935..."
3,130425,"[235407, 230067, 35265, 281005, 147734, 208935..."
4,93986,"[235407, 230067, 35265, 281005, 147734, 208935..."


In [57]:
recs = recs.explode('item_id')
recs.head(top_N + 2)

,user_id,item_id
0,38753,235407
0,38753,230067
0,38753,35265
0,38753,281005
0,38753,147734
0,38753,208935
0,38753,285394
0,38753,96052
0,38753,62715
0,38753,151190


In [58]:
recs['rank'] = recs.groupby('user_id').cumcount() + 1
recs.head(top_N + 2)

,user_id,item_id,rank
0,38753,235407,1
0,38753,230067,2
0,38753,35265,3
0,38753,281005,4
0,38753,147734,5
0,38753,208935,6
0,38753,285394,7
0,38753,96052,8
0,38753,62715,9
0,38753,151190,10


In [59]:
recs.dtypes

user_id     int64
item_id    object
rank        int64
dtype: object

In [60]:
recs['item_id'] = recs['item_id'].astype(np.int64)

In [61]:
test_recs = test.merge(recs, on = ['user_id', 'item_id'], how = 'inner')

In [62]:
test_recs.head()

,user_id,item_id,progress,rating,start_date,rank
0,102560,147734,0,NaN,2019-12-25,5
1,33152,147734,0,NaN,2019-12-25,5
2,55393,147734,1,NaN,2019-12-25,5
3,129781,147734,1,NaN,2019-12-25,5
4,57724,230067,74,NaN,2019-12-25,2


In [63]:
test_recs = test_recs.sort_values(by=['user_id', 'rank'])
test_recs.head()

,user_id,item_id,progress,rating,start_date,rank
46,474,235407,100,5.0,2019-12-25,1
18,1672,230067,12,NaN,2019-12-25,2
25,2345,208935,76,NaN,2019-12-25,6
61,9279,96052,100,NaN,2019-12-25,8
52,10260,35265,0,NaN,2019-12-25,3


### NDCG

Посчитайте метрику ndcg

In [ ]:
1/0

In [67]:
def ndcg_at_k(
    df: pd.DataFrame,
    user_col: str = 'user_id',
    rank_col: str = 'item_id',
    k: int = 1,
    log_base: int = 2,
) -> pd.Series:
    dcg = ((df[rank_col] <= k).astype(int) / log_at_base(df[rank_col] + 1, log_base))
    idcg = (1 / log_at_base(np.arange(1, k + 1) + 1, log_base)).sum()
    res = (
        pd.DataFrame({user_col: df[user_col], "ndcg": dcg / idcg})
        .groupby(user_col)["ndcg"]
        .sum()
    )
    return res

In [68]:
test_recs = test.set_index(['user_id', 'item_id']).join(recs.set_index(['user_id', 'item_id']))
test_recs['users_item_count'] = test_recs.groupby(level='user_id', sort=False)['rank'].transform(np.size)
test_recs['reciprocal_rank'] = 1 / test_recs['rank']
test_recs['reciprocal_rank'] = test_recs['reciprocal_rank'].fillna(0)
test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
test_recs.tail()

,,progress,rating,start_date,rank,users_item_count,reciprocal_rank,cumulative_rank
user_id,item_id,,,,,,,
129222,275441,24,NaN,2019-12-25,NaN,1.0,0.0,NaN
18067,69599,77,NaN,2019-12-25,NaN,1.0,0.0,NaN
76378,246837,93,NaN,2019-12-25,NaN,1.0,0.0,NaN
135722,267580,71,NaN,2019-12-25,NaN,1.0,0.0,NaN
92498,39822,35,NaN,2019-12-25,NaN,1.0,0.0,NaN


Посчитайте метрики по test (2019-12-25, 2019-12-26)
Precision@1 = 

Recall@1 = 

Precision@2 = 

Recall@2 = 

Precision@3 = 

Recall@3 = 

Precision@4 = 

Recall@4 = 

Precision@5 = 

Recall@5 = 

Precision@6 = 

Recall@6 = 

Precision@7 = 

Recall@7 = 

Precision@8 = 

Recall@8 = 

Precision@9 =

Recall@9 = 

Precision@10 = 

Recall@10 = 

MAP@10 = 

MRR = 

# ItemToItem модели

### Нужно запрограммировать BM25Okapi

In [71]:
import math

In [72]:

class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = len(corpus)
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer

        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word]+=1
                except KeyError:
                    nd[word] = 1

        self.avgdl = num_doc / self.corpus_size
        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        top_n = np.argsort(scores)[::-1][:n]
        return [documents[i] for i in top_n]



class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()


class BM25Plus(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, delta=1):
        # Algorithm specific parameters
        self.k1 = k1
        self.b = b
        self.delta = delta
        super().__init__(corpus, tokenizer)

    def _calc_idf(self, nd):
        """
        -
        """


    def get_scores(self, query):
        """-"""

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)




In [73]:
corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [74]:
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)



In [75]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

In [76]:
query = "How London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)

bm25.get_top_n(tokenized_query, corpus, n=1)

['How is the weather today?']

Возьмем BM25 из Implicit для рекомендаий

In [77]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise
import scipy.sparse as sp
from itertools import islice, cycle

In [78]:
df = pd.read_pickle('data/interactions_preprocessed.pickle')
df_users = pd.read_pickle('data/users_preprocessed.pickle')
df_items = pd.read_pickle('data/items_preprocessed.pickle')

In [79]:
df.shape, df_users.shape, df_items.shape

((1532998, 5), (142888, 3), (59599, 5))

In [80]:
users_inv_mapping = dict(enumerate(df['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

151600

In [81]:
items_inv_mapping = dict(enumerate(df['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

59599

In [82]:
df_items['title'] = df_items['title'].str.lower()
item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()
len(item_titles), item_titles[128115]

(59599, 'ворон-челобитчик')

In [83]:
title_items = df_items.groupby('title')['id'].agg(list)
title_items

title
# 20 восьмая                                                     [201623]
# duo                                                             [72582]
# me too. роман                                                  [171172]
# партия                                                         [224512]
#1917: человек из раньшего времени. библиотека «проекта 1917»    [230768]
                                                                   ...   
…чума на оба ваши дома!                                          [226481]
№ 12, или история одного прекрасного юноши                        [20979]
伦巴德人的故事                                                          [119226]
地球への旅                                                            [148400]
�baby blues�                                                      [98635]
Name: id, Length: 57289, dtype: object

In [84]:
title_count = title_items.map(len)
title_count.value_counts()

1     55708
2      1197
3       245
4        71
5        38
6        11
7         8
8         3
9         2
23        1
18        1
47        1
13        1
12        1
11        1
Name: id, dtype: int64

In [85]:
df['rating'] = np.array(df['rating'].values, dtype=np.float32)

df.loc[df['item_id'].isin([44681, 162716])].groupby('item_id').agg({
    'progress': np.size,
    'rating': ['mean'],
    'start_date': ['min', 'max'],
})

progress rating start_date           
            size   mean        min        max
item_id                                      
44681        353   4.56 2018-01-24 2019-12-20
162716        59   4.80 2018-01-25 2019-12-30

In [86]:
last_date = df['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [87]:
class TimeRangeSplit():
    """
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
    """
    def __init__(self, 
                 start_date, 
                 end_date=None, 
                 freq='D', 
                 periods=None, 
                 tz=None, 
                 normalize=False, 
                 closed=None, 
                 train_min_date=None,
                 filter_cold_users=True, 
                 filter_cold_items=True, 
                 filter_already_seen=True):
        
        self.start_date = start_date
        if end_date is None and periods is None:
            raise ValueError("Either 'end_date' or 'periods' must be non-zero, not both at the same time.")

        self.end_date = end_date
        self.freq = freq
        self.periods = periods
        self.tz = tz
        self.normalize = normalize
        self.closed = closed
        self.train_min_date = pd.to_datetime(train_min_date, errors='raise')
        self.filter_cold_users = filter_cold_users
        self.filter_cold_items = filter_cold_items
        self.filter_already_seen = filter_already_seen

        self.date_range = pd.date_range(
            start=start_date, 
            end=end_date, 
            freq=freq, 
            periods=periods, 
            tz=tz, 
            normalize=normalize, 
            closed=closed)

        self.max_n_splits = max(0, len(self.date_range) - 1)
        if self.max_n_splits == 0:
            raise ValueError("Provided parametrs set an empty date range.") 

    def split(self, 
              df, 
              user_column='user_id',
              item_column='item_id',
              datetime_column='date',
              fold_stats=False):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            train_min_mask = df_datetime >= self.train_min_date
        else:
            train_min_mask = df_datetime.notnull()

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        for start, end in pairwise(date_range):
            fold_info = {
                'Start date': start,
                'End date': end
            }
            train_mask = train_min_mask & (df_datetime < start)
            train_idx = df.index[train_mask]
            if fold_stats:
                fold_info['Train'] = len(train_idx)

            test_mask = (df_datetime >= start) & (df_datetime < end)
            test_idx = df.index[test_mask]
            
            if self.filter_cold_users:
                new = np.setdiff1d(
                    df.loc[test_idx, user_column].unique(), 
                    df.loc[train_idx, user_column].unique())
                new_idx = df.index[test_mask & df[user_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New users'] = len(new)
                    fold_info['New users interactions'] = len(new_idx)

            if self.filter_cold_items:
                new = np.setdiff1d(
                    df.loc[test_idx, item_column].unique(), 
                    df.loc[train_idx, item_column].unique())
                new_idx = df.index[test_mask & df[item_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New items'] = len(new)
                    fold_info['New items interactions'] = len(new_idx)

            if self.filter_already_seen:
                user_item = [user_column, item_column]
                train_pairs = df.loc[train_idx, user_item].set_index(user_item).index
                test_pairs = df.loc[test_idx, user_item].set_index(user_item).index
                intersection = train_pairs.intersection(test_pairs)
                test_idx = test_idx[~test_pairs.isin(intersection)]
                # test_mask = rd.df.index.isin(test_idx)
                if fold_stats:
                    fold_info['Known interactions'] = len(intersection)

            if fold_stats:
                fold_info['Test'] = len(test_idx)

            yield (train_idx, test_idx, fold_info)

    def get_n_splits(self, df, datetime_column='date'):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            df_datetime = df_datetime[df_datetime >= self.train_min_date]

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        return max(0, len(date_range) - 1)


In [88]:
last_date = df['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [89]:
cv = TimeRangeSplit(start_date=start_date, periods=folds+1)

cv.max_n_splits, cv.get_n_splits(df, datetime_column='start_date')

(7, 7)

In [90]:
folds_with_stats = list(cv.split(
    df, 
    user_column='user_id',
    item_column='item_id',
    datetime_column='start_date',
    fold_stats=True
))

folds_info_with_stats = pd.DataFrame([info for _, _, info in folds_with_stats])

In [91]:
fold_dates = [(info['Start date'], info['End date']) for _, _, info in folds_with_stats]
fold_dates


[(Timestamp('2019-12-24 00:00:00', freq='D'),
  Timestamp('2019-12-25 00:00:00', freq='D')),
 (Timestamp('2019-12-25 00:00:00', freq='D'),
  Timestamp('2019-12-26 00:00:00', freq='D')),
 (Timestamp('2019-12-26 00:00:00', freq='D'),
  Timestamp('2019-12-27 00:00:00', freq='D')),
 (Timestamp('2019-12-27 00:00:00', freq='D'),
  Timestamp('2019-12-28 00:00:00', freq='D')),
 (Timestamp('2019-12-28 00:00:00', freq='D'),
  Timestamp('2019-12-29 00:00:00', freq='D')),
 (Timestamp('2019-12-29 00:00:00', freq='D'),
  Timestamp('2019-12-30 00:00:00', freq='D')),
 (Timestamp('2019-12-30 00:00:00', freq='D'),
  Timestamp('2019-12-31 00:00:00', freq='D'))]

In [92]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    # код из прошлого заданния про метрики

In [93]:
train_idx, test_idx, info = folds_with_stats[0]

train = df.loc[train_idx]
test = df.loc[test_idx]
train.shape, test.shape

((1515946, 5), (2045, 5))

In [94]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [95]:
train_mat = get_coo_matrix(train).tocsr()
train_mat


<151589x59599 sparse matrix of type '<class 'numpy.float32'>'
	with 1515946 stored elements in Compressed Sparse Row format>

In [96]:
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender

In [97]:
cosine_model = CosineRecommender(K=10)
cosine_model.fit(train_mat.T) # 

  0%|          | 0/59599 [00:00<?, ?it/s]

In [98]:
md = BM25Recommender()
md.fit(train_mat.T) # 

  0%|          | 0/59599 [00:00<?, ?it/s]

In [99]:
top_N = 10
user_id = test['user_id'].iloc[0]
row_id = users_mapping[user_id]
print(f'Рекомендации для пользователя {user_id}, номер строки - {row_id}')

Рекомендации для пользователя 139942, номер строки - 10346


In [100]:
recs = cosine_model.recommend(row_id, train_mat, N=top_N, filter_already_liked_items=True)
recs = pd.DataFrame(recs, columns=['col_id', 'similarity'])
recs

,col_id,similarity
0,4341,0.297014
1,7353,0.220847
2,36593,0.215622
3,3802,0.188025
4,51215,0.145095
5,49085,0.128586
6,37852,0.102340
7,7873,0.101929
8,46769,0.100504
9,56270,0.100504


In [101]:
recs['item_id'] = recs['col_id'].map(items_inv_mapping.get)
recs['title'] = recs['item_id'].map(item_titles.get)
recs

,col_id,similarity,item_id,title
0,4341,0.297014,193358,#охотник на волков
1,7353,0.220847,125586,меч предназначения
2,36593,0.215622,203882,#имя для лис
3,3802,0.188025,90986,кровь эльфов
4,51215,0.145095,146180,крещение руси
5,49085,0.128586,264849,сплав закона
6,37852,0.102340,238155,владычица озера
7,7873,0.101929,7094,крещение огнем
8,46769,0.100504,240226,отпущение без грехов
9,56270,0.100504,1728,«злой город»


### Выберете наилучшую модель и параметры из Implcict с точки зрения максимизации map@10

# DSSM Torch

Torch

In [72]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [73]:
from scipy import sparse
import typing as tp
from tqdm.notebook import tqdm

In [74]:
df.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [75]:
cv.get_n_splits(df, datetime_column='start_date')

7

In [76]:
%%time
folds = list(cv.split(df, datetime_column='start_date'))
pd.DataFrame([stats for _, _, stats in folds])

CPU times: user 8.72 s, sys: 1.44 s, total: 10.2 s
Wall time: 10.2 s


,Start date,End date
0,2019-12-24,2019-12-25
1,2019-12-25,2019-12-26
2,2019-12-26,2019-12-27
3,2019-12-27,2019-12-28
4,2019-12-28,2019-12-29
5,2019-12-29,2019-12-30
6,2019-12-30,2019-12-31


In [77]:
df['user_id'] = df['user_id'].astype('category')
df['item_id'] = df['item_id'].astype('category')

In [78]:
class DSSM(nn.Module):
    
    def __init__(
        self,
        uf_dim: int,
        uf1_dim: int,
        uf2_dim: int,
        if_dim: int,
        if1_dim: int,
        if2_dim: int,
        final_dim: int,
        dropout: float,
    ):
        super().__init__()
        
        self.user_seq = nn.Sequential(
            nn.Linear(uf_dim, uf1_dim),
            nn.BatchNorm1d(uf1_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(uf1_dim, uf2_dim),
            nn.BatchNorm1d(uf2_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(uf2_dim, final_dim),
            nn.BatchNorm1d(final_dim),
            nn.ReLU(),
        )
        self.item_seq = nn.Sequential(
            nn.Linear(if_dim, if1_dim),
            nn.BatchNorm1d(if1_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(if1_dim, if2_dim),
            nn.BatchNorm1d(if2_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(if2_dim, final_dim),
            nn.BatchNorm1d(final_dim),
            nn.ReLU(),
        )        
        self.cosine = nn.CosineSimilarity()
    
    def forward(
        self,
        user_features,
        item_features,
    ):
        user_vec = self.user_seq(user_features)
        item_vec = self.item_seq(item_features)
        sim = self.cosine(user_vec, item_vec)
        return sim

In [79]:
def csr_to_tensor(csr: sparse.csr_matrix, flatten=False) -> torch.Tensor:
    arr = csr.toarray()
    if flatten:
        arr = arr.reshape(-1)
    return torch.from_numpy(arr)


def make_id_csr(ids: pd.Series, n_cols: int) -> sparse.csr_matrix:
    csr = sparse.csr_matrix(
        (
            np.ones(len(ids)),
            (
                np.arange(len(ids)),
                ids,
            )
        ),
        shape=(len(ids), n_cols),
    )
    return csr

def binary_accuracy(true: torch.Tensor, pred: torch.Tensor) -> torch.Tensor:
    predicted_labels = torch.round(torch.sigmoid(pred))
    correct = (predicted_labels == true).float()
    acc = correct.sum(axis=0) / len(correct)
    return acc



def process_epoch(
    net: nn.Module,
    loader: DataLoader,  # type: ignore
    optimizer: optim.Optimizer,
    criterion: nn.modules.loss._Loss,
    device: torch.device,
    process: str, # "train", "eval"
) -> tp.Tuple[float, float, np.ndarray, np.ndarray]:
    epoch_loss = 0.
    epoch_acc = 0
    n_batches = len(loader)
    all_true = []
    all_predictions = []

    if process == "train":
        net.train()
    elif process == "eval":
        net.eval()

    with torch.set_grad_enabled(process == "train"):
        for batch in tqdm(loader, total=n_batches):
            u_batch, i_batch, y_batch = [b.to(device) for b in batch]
            pred = net(u_batch, i_batch)
            loss = criterion(pred, y_batch.type_as(pred))
            acc = binary_accuracy(y_batch, pred)

            if process == "train":
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.detach().cpu().numpy()
            
            all_true.append(y_batch.detach().cpu().numpy())
            all_predictions.append(torch.sigmoid(pred).detach().cpu().numpy())

    return (
        epoch_loss / n_batches, 
        epoch_acc / n_batches, 
        np.concatenate(all_true, axis=0),
        np.concatenate(all_predictions, axis=0),
    )

In [80]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [81]:
train_idx, test_idx, _ = folds[0]

df_train = df.loc[train_idx]
#df_train['user_id'].cat.remove_unused_categories(inplace=True)
#df_train['item_id'].cat.remove_unused_categories(inplace=True)
df_test = df.loc[test_idx]

In [82]:
unq_users = np.unique(
    np.concatenate(
        (
            df_train['user_id'].values,
            df_test['item_id'].values,
        )
    )
)
unq_items = np.unique(
    np.concatenate(
        (
            df_train['user_id'].values,
            df_test['item_id'].values,
        )
    )
)

In [83]:
df_train.dtypes

user_id             category
item_id             category
progress                int8
rating               float32
start_date    datetime64[ns]
dtype: object

In [84]:
class RecDataset(Dataset):
    
    def __init__(
        self,
        interactions_df: pd.DataFrame,
        unq_u_ids=None,
        unq_i_ids=None,
        neg_share=0.5,
    ):
        u_ids_pos = interactions_df['user_id'].cat.codes.values
        i_ids_pos = interactions_df['item_id'].cat.codes.values
        
        if unq_u_ids is None:
            unq_u_ids = np.unique(u_ids_pos)
        if unq_i_ids is None:
            unq_i_ids = np.unique(i_ids_pos)
            
        assert np.isin(u_ids_pos, unq_u_ids).all()
        assert np.isin(i_ids_pos, unq_i_ids).all()
        
        n_negs = int(len(interactions_df) * neg_share / (1 - neg_share))
        
        # take random, do not check (can be positives here, but probability is very low)
        u_ids_neg = np.random.choice(unq_u_ids, n_negs)
        i_ids_neg = np.random.choice(unq_i_ids, n_negs)
        
        user_features = make_id_csr(np.concatenate((u_ids_pos, u_ids_neg)), n_cols=unq_u_ids.size)
        item_features = make_id_csr(np.concatenate((i_ids_pos, i_ids_neg)), n_cols=unq_i_ids.size)
        
        self.user_features = user_features.astype(np.float32)
        self.item_features = item_features.astype(np.float32)
        self.y = np.concatenate((np.ones(len(interactions_df)), np.zeros(n_negs)))
        
    def __getitem__(self, index: int):
        u_f = csr_to_tensor(self.user_features[index], flatten=True)
        i_f = csr_to_tensor(self.item_features[index], flatten=True)
        y = self.y[index]
        return u_f, i_f, y
        
    def __len__(self):
        return self.user_features.shape[0]
    


In [85]:
train_idx, test_idx, _ = folds[0]

df_train = df.loc[train_idx]
df_train['user_id'].cat.remove_unused_categories(inplace=True)
df_train['item_id'].cat.remove_unused_categories(inplace=True)

df_test = df.loc[test_idx]
df_test['user_id'].cat.remove_unused_categories(inplace=True)
df_test['item_id'].cat.remove_unused_categories(inplace=True)

unq_users = np.unique(
    np.concatenate(
        (
            df_train['user_id'].cat.codes.values,
            df_test['user_id'].cat.codes.values,
        )
    )
)
unq_items = np.unique(
    np.concatenate(
        (
            df_train['item_id'].cat.codes.values,
            df_test['item_id'].cat.codes.values,
        )
    )
)

train_data = RecDataset(df_train, unq_u_ids=unq_users, unq_i_ids=unq_items)
test_data = RecDataset(df_test, unq_u_ids=unq_users, unq_i_ids=unq_items)

train_loader = DataLoader(
    train_data,
    batch_size=128,
    num_workers=8,
    pin_memory=True,
    shuffle=True,
)
test_loader = DataLoader(
    test_data,
    batch_size=128,
    num_workers=4,
    pin_memory=True,
    shuffle=False,
)

In [86]:
net = DSSM(
    uf_dim=unq_users.size,
    uf1_dim=32, #256
    uf2_dim=16, #128
    if_dim=unq_items.size,
    if1_dim=32, #256
    if2_dim=16, #128
    final_dim=8, # 64
    dropout=0.5,
)
net.to(device)

DSSM(
  (user_seq): Sequential(
    (0): Linear(in_features=151589, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=16, out_features=8, bias=True)
    (9): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (item_seq): Sequential(
    (0): Linear(in_features=59599, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6

In [87]:
optimizer = optim.Adam(net.parameters())
criterion = nn.BCEWithLogitsLoss().to(device)

In [88]:
%%time
train_losses = []
valid_losses = []
best_valid_loss = float('inf')
best_model_state: tp.Dict[str, torch.Tensor] = {}
for epoch in range(1):
    print(f"Epoch {epoch + 1} started")

    train_loss, train_acc, train_true, train_pred = process_epoch(net, train_loader, optimizer, criterion, device, "train")
    valid_loss, valid_acc, valid_true, valid_pred = process_epoch(net, test_loader, optimizer, criterion, device, "eval")
    
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_model_state = net.state_dict()
    
#     train_aucs = roc_auc_many(train_true, train_pred)
#     valid_aucs = roc_auc_many(valid_true, valid_pred)
    
    print(f"Epoch: {epoch + 1:02}")
    print(
        f"\n\t Train Loss: {train_loss:.3f}"
        f"\n\t Train Accuracy: {train_acc.round(3)}"
#         f"\n\t Train AUC: {np.array(train_aucs).round(3)}"
    )
    print(
        f"\n\t Valid Loss: {valid_loss:.3f}"
        f"\n\t Valid Accuracy: {valid_acc.round(3)}"
#         f"\n\t Valid AUC: {np.array(valid_aucs).round(3)}"
    )

net.load_state_dict(best_model_state)

Epoch 1 started


  0%|          | 0/23687 [00:00<?, ?it/s]

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/opt/anaconda2/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/anaconda2/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/anaconda2/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/torch/multiprocessing/reductions.py", line 289, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/anaconda2/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/opt/anaconda2/lib/python3.7/multiprocessing/resource_sharer.py", line 87, in ge

KeyboardInterrupt: 

# Алгоритмы ранжирования

In [102]:
import numpy as np
import pandas as pd
import lightgbm

In [103]:
df = pd.DataFrame({
    "query_id":[i for i in range(100) for j in range(10)],
    "var1":np.random.random(size=(1000,)),
    "var2":np.random.random(size=(1000,)),
    "var3":np.random.random(size=(1000,)),
    "relevance":list(np.random.permutation([0,0,0,0,0, 0,0,0,1,1]))*100
})

In [104]:
df.head()

,query_id,var1,var2,var3,relevance
0,0,0.145919,0.318222,0.004795,0
1,0,0.570852,0.498379,0.647453,0
2,0,0.638582,0.002770,0.079687,0
3,0,0.846366,0.698434,0.507535,0
4,0,0.529181,0.738735,0.548863,0


In [105]:
df.tail()

,query_id,var1,var2,var3,relevance
995,99,0.927659,0.025131,0.539747,1
996,99,0.595341,0.333927,0.297698,1
997,99,0.630101,0.871618,0.008137,0
998,99,0.722011,0.919749,0.133257,0
999,99,0.747857,0.221893,0.212690,0


In [108]:
train_df = df[:800]  # first 80%
validation_df = df[800:]  # remaining 20%

qids_train = train_df.groupby("query_id")["query_id"].count().to_numpy()
X_train = train_df.drop(["query_id", "relevance"], axis=1)
y_train = train_df["relevance"]

qids_validation = validation_df.groupby("query_id")["query_id"].count().to_numpy()
X_validation = validation_df.drop(["query_id", "relevance"], axis=1)
y_validation = validation_df["relevance"]

In [109]:
qids_train

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [110]:
qids_validation

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10])

In [111]:
X_validation.head()

,var1,var2,var3
800,0.653470,0.994187,0.585130
801,0.686948,0.087938,0.007318
802,0.791950,0.660912,0.950166
803,0.658954,0.594713,0.059344
804,0.152590,0.285497,0.897389


In [112]:
y_validation.head()

800    0
801    0
802    0
803    0
804    0
Name: relevance, dtype: int64

In [113]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [114]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=10,
    verbose=10,
)

[10]	valid_0's ndcg@10: 0.552886
[20]	valid_0's ndcg@10: 0.519568
[30]	valid_0's ndcg@10: 0.514397


KeyboardInterrupt: 

### Реализуйте lambdamart model

In [103]:
import numpy as np
from lightgbm import LGBMRanker

N = 20
y = np.arange(N)
X = np.random.normal(size=(N, 2))
X[:, 1] = y[0:]

model = LGBMRanker(min_data=1, min_data_in_bin=1)
model.fit(X, y, group=[N])
print(model.predict(X))

[-9.11921696 -9.06961912 -9.09686847 -9.05784061 -9.06402035 -8.85883171
 -8.80634888 -8.61861135 -8.26913952 -7.7986417  -7.26218883 -6.59235805
 -5.79645127 -4.8224706  -3.73366565 -2.40071506 -0.7663117   1.41545657
  4.45896242  8.68988917]


In [115]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
import pandas as pd

In [116]:
from catboost.datasets import msrank_10k

In [117]:
train_df = pd.read_csv('msrank_train.csv', index_col=0)

In [118]:
test_df = pd.read_csv('msrank_test.csv', index_col=0)

In [119]:
train_df.shape

(10000, 138)

In [121]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2.0,1,3,3,0,0,3,1.0,1.0,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.0
1,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.0,1,3,0,2,0,3,1.0,0.0,0.666667,...,45,3,1,124,3344,14,67,0,0,0.0
3,2.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.0
4,1.0,1,3,0,3,0,3,1.0,0.0,1.000000,...,64,5,7,256,49697,1,13,0,0,0.0


In [120]:
test_df.shape

(10000, 138)

### Ranking problem

Введем обозначения:
* $Q = \{q_1, \dots, q_n\}$ список запросов
* $D_q = \{d_{q1}, \dots, d_{qm}\}$ -- набор документов, полученных для группы $q$
* $L_q = \{l_{q1}, \dots, l_{qm}\}$ -- метки релевантности для объектов из набор $D_q$

Каждый обьект $d_{qi}$ представлен в векторном пространстве признаков, описывающих связи между группой и объектом.

Таким образом, каждая группа связана с набором объектов. Например, группа - это запрос, а объект - это документ, если мы ранжируем документы по поисковому запросу.

Цель состоит в том, чтобы изучить функцию ранжирования $ f = f (d_ {qi}) $, такую, чтобы ранжирование объектов $ d_ {qi} $ для всех групп из $ Q $ основывалось на их оценках $ x_ {qi} = f (d_ {qi}) $ максимально приближенных к идеальному рейтингу согласно $ l_ {qi} $.

### Ranking quality metrics:
* __Precision__
    $$ \mbox{P}=\frac{|\{\mbox{relevant docs}\}\cap\{\mbox{retrieved docs}\}|}{|\{\mbox{retrieved docs}\}|} $$
* __Recall__
    $$ \mbox{R}=\frac{|\{\mbox{relevant docs}\}\cap\{\mbox{retrieved docs}\}|}{|\{\mbox{relevant docs}\}|} $$
    
    Обозначение $ @ k $ означает, что метрика рассчитывается для первых $ k $ документов из списка ранжирования.

     Например, если 1,2,5,7,9 - это ранги релевантных документов (перечисления начинаются с номера 1 из десяти полученных, то $ P @ 5 $ будет $\frac{3}{5}$.

* __Mean average precision (MAP)__
    $$\frac{1}{|Q|}\sum_{q \in Q} \frac{1}{|\mbox{relevant docs in } D_q|} \sum_{k} P@k(q) \times rel(q, k) $$
    
    Где $rel(q, k)$ - метка релевантности документа на k-й позиции в нашем рейтинге $ D_q $. Этот показатель вычисляет среднюю точность для запроса, взвешенного с учетом релевантности документов, а затем вычисляет среднее значение между всеми запросами.
    
* __Discounted cumulative gain (DCG)__
    $$\sum_{k=1}^{mq} \frac{2 ^ {l_{qk}}}{\log_2(k+1)}$$
    
    Эта метрика учитывает поведение пользователя: внимание пользователя находится наверху, а затем нелинейно уменьшается до конца.
    
* __NDCG__ - нормализованный DCG = DCG $ ~ / ~ $ IDCG, где IDCG - максимально возможное значение DCG с заданным набором меток релевантности.

* __AverageGain__ - представляет среднее значение значений метки для объектов с определенными значениями верхней метки.
* __[PFound](https://tech.yandex.com/catboost/doc/dg/references/pfound-docpage/#pfound)__
    
Больше на wiki: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Параметр $ @ k $ для каждой метрики можно указать через параметр метрики «top», например «NDCG: top = 10» будет означать NDCG @ 10.

In [123]:
#from catboost.datasets import msrank_10k
#train_df, test_df = msrank_10k()

X_train = train_df.drop(['0', '1'], axis=1).values
y_train = train_df['0'].values
queries_train = train_df['1'].values

X_test = test_df.drop(['0', '1'], axis=1).values
y_test = test_df['0'].values
queries_test = test_df['1'].values

In [124]:
num_documents = X_train.shape[0]
print(num_documents)

10000


Число фичей

In [125]:
X_train.shape[1]

136

__Relevance labels statistics__

0 - неактуально, 4 - очень актуально. В таблице представлено количество документов для каждого значения.

In [126]:
from collections import Counter
Counter(y_train).items()

dict_items([(2.0, 1326), (0.0, 5481), (1.0, 3000), (3.0, 142), (4.0, 51)])

In [127]:
max_relevance = np.max(y_train)
y_train /= max_relevance
y_test /= max_relevance

In [128]:
X_train

array([[3., 3., 0., ..., 0., 0., 0.],
       [3., 0., 3., ..., 0., 0., 0.],
       [3., 0., 2., ..., 0., 0., 0.],
       ...,
       [2., 0., 2., ..., 0., 0., 0.],
       [2., 0., 1., ..., 0., 0., 0.],
       [2., 1., 1., ..., 0., 0., 0.]])

In [130]:
num_queries = np.unique(queries_train).shape[0]
num_queries

87

In [131]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

### Можем создать pools из файлов

In [138]:
data_dir = './msrank'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

train_file = os.path.join(data_dir, 'train.csv')
test_file = os.path.join(data_dir, 'test.csv')

train_df.to_csv(train_file, index=False, header=False)
test_df.to_csv(test_file, index=False, header=False)

In [139]:
description_file = os.path.join(data_dir, 'dataset.cd')
with open(description_file, 'w') as f:
    f.write('0\tLabel\n')
    f.write('1\tQueryId\n')

### <span style="color:#ce2029">Attention:</span> Все обьекты должны быть сгруппированы по group_id

Например, если набор данных состоит из пяти документов
\ [d1, d2, d3, d4, d5 \] с соответствующими запросами \ [q1, q2, q2, q1, q2 \], тогда набор данных должен выглядеть так:

$$\begin{pmatrix}
    d_1, q_1, f_1\\
    d_4, q_1, f_4\\
    d_2, q_2, f_2\\
    d_3, q_2, f_3\\
    d_5, q_2, f_5\\
\end{pmatrix} \hspace{6px} \texttt{or} \hspace{6px}
\begin{pmatrix}
    d_2, q_2, f_2\\
    d_3, q_2, f_3\\
    d_5, q_2, f_5\\
    d_1, q_1, f_1\\
    d_4, q_1, f_4\\
\end{pmatrix}$$

где $f_i$ это вектор фичей для i-го документа.

### Сведение проблемы к задаче машинного обучения

Первая и самая простая идея - попытаться предсказать релевантность документа $ l_q $, минимизируя RMSE.

$$\frac{1}{N}\sqrt{ \sum_q \sum_{d_{qk}} \left(f(d_{qk}) - l_{qk} \right)^2 }$$

In [132]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [133]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    #model.fit(train_pool, eval_set=test_pool, plot=True)
    model.fit(train_pool, eval_set=test_pool, plot=False)
    return model

In [134]:
model = fit_model('RMSE', {'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})

/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/catboost/core.py:5765: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


### Параметр весов групп
Предположим, мы знаем, что одни запросы для нас важнее других. <br/>
Слово «важность» используется здесь с точки зрения точности или качества прогнозов CatBoostRanker для заданных запросов. <br/>
Вы можете передать эту дополнительную информацию алгоритму с помощью параметра group_weights. <br/>
Под капотом CatBoostRanker использует эти веса в функции потерь, просто умножая их на групповое слагаемое. <br/>
Таким образом, чем больше вес, тем больше внимания будет уделяться запросу. <br/>
Приведем пример процедуры обучения со случайными весами запросов.

In [135]:
def create_weights(queries):
    query_set = np.unique(queries)
    query_weights = np.random.uniform(size=query_set.shape[0])
    weights = np.zeros(shape=queries.shape)
    
    for i, query_id in enumerate(query_set):
        weights[queries == query_id] = query_weights[i]
    
    return weights
    

train_with_weights = Pool(
    data=X_train,
    label=y_train,
    group_weight=create_weights(queries_train),
    group_id=queries_train
)

test_with_weights = Pool(
    data=X_test,
    label=y_test,
    group_weight=create_weights(queries_test),
    group_id=queries_test
)

fit_model(
    'RMSE', 
    additional_params={'train_dir': 'RMSE_weigths'}, 
    train_pool=train_with_weights,
    test_pool=test_with_weights
)


Поэксперементируйте с весами, задав их сокореллировано релевантности, сокореллировано частоте

## Особый случай: прогноз топ-1

Когда-нибудь вы можете столкнуться с проблемой $ - $, вам нужно будет предсказать один из наиболее релевантных объектов для данного запроса. <br/>
Для этого в CatBoostRanker есть режим __QuerySoftMax__.

Предположим, что у вашего набора данных бинарный таргет: 1 $ - $ означает лучший документ для запроса, 0 $ - $ другие. <br/>
Мы максимально увеличим вероятность того, что станем лучшим документом для данного запроса. <br/>
Набор данных MSRANK не содержит двоичных меток, но, например, для метода __QuerySoftMax__ мы конвертируем его в этот формат, <br/> выбирая лучший документ для каждого запроса.

In [136]:
def get_best_documents(labels, queries):
    query_set = np.unique(queries)
    num_queries = query_set.shape[0]
    by_query_arg_max = {query: -1 for query in query_set}
    
    for i, query in enumerate(queries):
        best_idx = by_query_arg_max[query]
        if best_idx == -1 or labels[best_idx] < labels[i]:
            by_query_arg_max[query] = i
    
    binary_best_docs = np.zeros(shape=labels.shape)
    for arg_max in by_query_arg_max.values():
        binary_best_docs[arg_max] = 1.
        
    return binary_best_docs

In [140]:
best_docs_train = get_best_documents(y_train, queries_train)
best_docs_test = get_best_documents(y_test, queries_test)

train_with_weights = Pool(
    data=X_train,
    label=best_docs_train,
    group_id=queries_train,
    group_weight=create_weights(queries_train)
)

test_with_weights = Pool(
    data=X_test,
    label=best_docs_test,
    group_id=queries_test,
    group_weight=create_weights(queries_test)
)

fit_model(
    'QuerySoftMax',
    additional_params={'custom_metric': 'AverageGain:top=1'},
    train_pool=train_with_weights,
    test_pool=test_with_weights
)

### Уменьшение проблемы, шаг 2

Теперь посмотрим на пример релевантности документов:

$$ 
    \begin{align}
    labels(q_1) &= \begin{bmatrix}
           4 \\
           3 \\
           3 \\
           1
         \end{bmatrix},
    labels(q_2) &= \begin{bmatrix}
           2 \\
           1 \\
           1 \\
           0
         \end{bmatrix}
   \end{align}
$$

Это означает, что с функцией потерь RMSE мы уделяем больше внимания q1, чем q2.

Чтобы избежать этой проблемы, мы вводим в RMSE коэффициент $ c_q $, который зависит только от запроса (и если факт равен среднему значению разницы между предсказанием и меткой).

$$\frac{1}{N}\sqrt{ \sum_q \sum_{d_{qk}} \left(f(d_{qk}) - l_{qk} - \color{red}{c_{q}} \right)^2 }$$

In [54]:
fit_model('QueryRMSE')

### Уменьшение проблемы, шаг 3

Поскольку целью ранжирования является прогнозирование списка документов (который может быть сгенерирован на основе заданных релевантностей документов), функция потерь RMSE не учитывает отношения между документами: первый лучше, чем второй, второй лучше, чем третий и пятый и т. Д. .

Мы можем легко внести эту информацию в функцию потерь, сведя проблему не к регрессии, а к классификации для двух документов. $ (D_i, d_j) $ - делает $ i $ ый лучше, чем $ j $ ый, или нет.

Таким образом, мы минимизируем отрицательную логарифмическую вероятность:

$$ - \sum_{i, j \in Pairs} \log \left (\frac {1} {1 + \exp {- (f (d_i) - f (d_j))}} \right) $$

Методы, основанные на парных сравнениях, называются __pairwise__ в CatBoostRanker, эта цель называется __PairLogit__.

Нет необходимости изменять набор данных, которые CatBoost генерирует для нас. Количество генерируемых пар, управляемых параметром max_size.

In [ ]:
fit_model('PairLogit')

Также мы можем указать пары напрямую. Это можно сделать двумя способами:

1. Двумерная матрица с shape = (num_pairs, 2) $ \rightarrow $ (Win_id, loser_id): list, numpy.array, pandas.DataFrame.
2. Второй путь к входному файлу, содержащему описания пар:
     * Формат строки: $ \texttt {[индекс победителя, индекс проигравшего, вес пары]} $

In [141]:
def read_groups(file_name):
    groups = {}
    group_ids = []

    with open(file_name) as f:
        for doc_id, line in enumerate(f):
            line = line.split(',')[:2]
            
            label, query_id = float(line[0]), int(line[1])
            if query_id not in groups:
                groups[query_id] = []
            groups[query_id].append((doc_id, label))

            group_ids.append(query_id)

    return groups, group_ids
            
train_groups, train_group_ids = read_groups(train_file)
assert num_queries == len(train_groups)

In [142]:
pairs = []

for group in train_groups.values():
    for i in range(len(group)):
        for j in range(i, len(group)):
            if i == j:
                continue
            doc_i, relevance_i = group[i]
            doc_j, relevance_j = group[j]
            if relevance_i < relevance_j:
                pairs.append((doc_j, doc_i))
            else:
                pairs.append((doc_i, doc_j))
                
pairs_file = os.path.join(data_dir, 'pairs.csv')

with open(pairs_file, 'w') as f:
    for pair in pairs:
        f.write(str(pair[0]) + '\t' + str(pair[1]) + '\t1\n')

In [143]:
pool1 = Pool(data=X_train, label=y_train, group_id=train_group_ids, pairs=pairs)
pool2 = Pool(data=train_file, column_description=description_file, pairs=pairs_file, delimiter=',')

### шаг 3.1

In [ ]:
fit_model('PairLogitPairwise')

### Уменьшение проблемы, шаг 4

Предыдущая функция потерь напрямую минимизировала количество пар $ (d_i, d_j) $, где $ l_i> l_j $, но $ f (d_i) <f (d_j) $, просто указала количество неправильно размещенных документов.

Поскольку внимание пользователя уделяется первым документам и низким - последним, неправильное переключение первых двух документов и двух последних имеет разную стоимость.

На шагах 3 и 3.1 пользователь может установить вес пары.

Метод __YetiRank__ учитывает этот эффект и генерирует веса для пар в соответствии с их положением ([paper] (https://cache-mskstoredata08.cdn.yandex.net/download.yandex.ru/company/to_rank_challenge_with_yetirank.pdf)).

$$ - \sum_{i,j \in Pairs} \color{red}{w_{ij}} \log \left( \frac{1}{1 + \exp{-(f(d_i) - f(d_j))}} \right) $$

In [ ]:
fit_model('YetiRank')

### Шаг 4.1

Как и в шаге 3.1, __YetiRankPairwise__ медленнее, чем __YetiRank__, но дает более точные результаты.

In [ ]:
fit_model('YetiRankPairwise')

In [60]:
widget = MetricVisualizer(['RMSE', 'QueryRMSE', 'PairLogit', 'PairLogitPairwise', 'YetiRank', 'YetiRankPairwise'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

KeyboardInterrupt: 

In [61]:
widget = MetricVisualizer(['RMSE', 'QueryRMSE'])
widget.start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

### Простая классификация

Очень быстро $ \rightarrow $ очень медленно; Простой метод $ \rightarrow $ сложный метод; Низкое качество $ \rightarrow $ высокое качество.

1. RMSE
2. QueryRMSE
3. PairLogit
4. PairLogitPairwise
5. YetiRank
6. YetiRankPairwise

Помимо нашей классификации, качество конкретного метода может зависеть от вашего набора данных.

Посмотрите на метрику NDCG метода YetiRank $ - $ она недостаточно приспособлена.

In [144]:
fit_model('YetiRank', {'train_dir': 'YetiRank-lr-0.3', 'learning_rate': 0.3})

KeyboardInterrupt: 

In [ ]:
widget = MetricVisualizer(['YetiRank', 'YetiRank-lr-0.3'])
widget.start()

## Регрессия против классификации против LTR
Все они обучаются c учителем. Но целевые переменные различаются. Алгоритм обучения будет отличаться тем, как мы математически формулируем цель обучения.

**Регрессия**
Мы пытаемся изучить функцию f (x) с учетом признака x, чтобы предсказать действительное значение y∈ℝ.

Пример: я хочу спрогнозировать средний баланс CASA в следующие 7 дней для учетной записи.

**Классификация**
Мы пытаемся изучить функцию f (x) с учетом признака x, чтобы предсказать набор дискретных целочисленных меток y∈ {1,2, ..., N} с N -классами.

**Пример**: я хочу предсказать, будет ли продлен данный FD (фиксированный депозит) по истечении текущего срока.

**Обучение ранжированию**
Мы пытаемся изучить функцию f (q, D), учитывая запрос q и соответствующий список элементов D, чтобы предсказать порядок (ранжирование) всех элементов в списке.

## Классическая проблема в LTR

**web-search ranking**

Учитывая поисковый запрос, ранжируйте релевантность результирующих совпадающих URL-адресов документов, чтобы в первую очередь пользователю был представлен более релевантный документ.

Более формально мы изображаем вышеуказанную проблему как следующую задачу:

Учитывая запрос q и результирующие n документов D = d1, d2, ..., dn, мы хотели бы изучить функцию f, такую, чтобы f (q, D) предсказывала релевантность любого данного документа, связанного с запрос. В идеале функция f (q, D) должна возвращать упорядоченный список документов D , ранжированных от наиболее до наименее релевантных для данного запроса q.

Популярные наборы данных веб-поиска для крупномасштабного теста LTR:

Наборы данных Microsoft LTR

Наборы данных Yahoo LTR

## Неклассические проблемы в LTR
LTR - это общий подход к решению задачи ранжирования. Вот еще несколько примеров, помимо рейтинга в веб-поиске. Обратите внимание, что не все из них, на первый взгляд, представляют собой задачу ранжирования.

**Рекомендательная система** (Вычисление персональный рейтинг предпочтений продуктов)

**Выбор портфеля акций** (Вычисление доходности акций)

**Автоответчик на сообщение** (Определение рейтинга лучших кандидатов в рекомендациях по электронной почте / ответу на сообщение)

**Изображение в текст** (Решение контекстной функции лучшего кандидата) 

## Типы  LTR алгоритмов:
- Pointwise
- Pairwise
- Listwise

Они отличаются тем, как мы формулируем **функцию потерь** в основной задаче машинного обучения.



**Задача ранжирования**

Учитывая запрос q и результирующий документ n D = d1, d2, ..., dn, мы хотели бы изучить функцию f, такую, чтобы f (q, D) предсказывала релевантность любого данного документа, связанного с запрос.

## pairwise LTR
При попарном подходе мы все еще пытаемся изучить функцию точечной оценки f (q, di), однако наши обучающие примеры теперь состоят из пар документов в одном запросе:

x1: q1, (d1, d2)
х2: q2, (d3, d4)
x3: q2, (d3, d5)
x4: q2, (d4, d5)

При такой настройке можно получить новый набор попарных БИНАРНЫХ меток, просто сравнив индивидуальную оценку релевантности в каждой паре. Например, для первого запроса q1, если y1 = 0 (совершенно неактуально) для d1 и y2 = 3 (очень актуально) для d2, то у нас есть новая метка y1 <y2 для пары документов (d1, d2). **Теперь проблема превратилась в задачу обучения бинарной классификации.**

Для того чтобы попарно изучить поточечную функцию f (q, di), мы моделируем разницу в баллах вероятностно:
 
$Pr(i \succ j) \equiv \frac{1}{1 + exp^{-(s_i - s_j)}}$
Проще говоря, если документ i соответствует лучше, чем документ j (который мы обозначаем как i≻j), то вероятность того, что функция оценки получит оценку f (q, di) = si выше, чем f (q, dj) = sj должен быть близок к 1. Другими словами, модель пытается научиться, учитывая запрос, как оценить пару документов, чтобы более релевантный документ получил более высокую оценку.

Плюсы:
Модель учится ранжировать напрямую, хотя и только попарно, но теоретически она может приблизиться к производительности общей задачи ранжирования при наличии N документов в согласованном списке.
Нам не нужны явные точечные метки. Требуются только парные предпочтения. Это преимущество, потому что иногда мы можем сделать вывод о парном предпочтении только на основе собранных данных о поведении пользователей.
Минусы:
Сама функция скоринга по-прежнему является точечной, а это означает, что относительная информация в пространстве функций среди разных документов по одному и тому же запросу все еще не используется полностью.

**Задача ранжирования**

Учитывая запрос q и результирующий документ n D = d1, d2, ..., dn, мы хотели бы изучить функцию f, такую, чтобы f (q, D) предсказывала релевантность любого данного документа, связанного с запрос.

## Listwise LTR
ListNet - это первый предложенный списочный подход. Здесь мы объясняем, как он подходит к задаче ранжирования.

ListNet основан на концепции вероятности перестановки с учетом ранжированного списка. Снова мы предполагаем, что есть функция точечной оценки f (q, di), используемая для оценки и, следовательно, ранжирования данного списка элементов. Но вместо моделирования вероятности попарного сравнения с использованием разницы оценок теперь мы хотели бы смоделировать вероятность всех результатов ранжирования.

x1: q1, (d1, d2)

x2: q2, (d3, d4, d5)

**вероятность перестановки**

Обозначим π как конкретную перестановку данного списка длины n, ϕ (si) = f (q, di) как любую возрастающую функцию оценки si для данного запроса q и документа i. Вероятность наличия перестановки π может быть записана как

$Pr(\pi) = \prod_{i=1}^n \frac{\phi(s_i)}{\sum_{k=i}^n\phi(s_k)}$

Для иллюстрации, учитывая список из 3 элементов, вероятность возврата перестановки s1, s2, s3 рассчитывается как: 

$Pr(\pi = \{s_1, s_2, s_3\}) = \frac{\phi(s_1)}{\phi(s_1) + \phi(s_2) + \phi(s_3)} \cdot \frac{\phi(s_2)}{\phi(s_2) + \phi(s_3)} \cdot \frac{\phi(s_3)}{\phi(s_3)}$

Из-за вычислительной сложности ListNet упрощает задачу, рассматривая только первую вероятность данного элемента. Вероятность первого попадания объекта i равна сумме вероятностей перестановок, в которых объект i занимает первое место. В самом деле, вероятность первого попадания объекта i может быть записана как:

$Pr(i) = \frac{\phi(s_i)}{\sum_{k=1}^n \phi(s_k)}$

Теперь, учитывая любые два списка ранжирования, представленные вероятностями первого числа, мы можем измерить их разницу с помощью перекрестной энтропии. Затем мы можем построить алгоритм машинного обучения, который минимизирует перекрестную энтропию.

Для выбора функции ϕ (⋅) это может быть так же просто, как просто экспоненциальная функция. В самом деле, когда ϕ (⋅) экспоненциально, а длина списка равна двум, решение сведется к попарной модели, которую мы только что описали в предыдущем разделе.

- Плюсы:

     - Теоретически обоснованное решение подойти к задаче ранжирования.

- Минусы:

     - Дорогостоящие вычисления в теоретической форме, поэтому на практике используются несколько приближений. (Например, использование первой вероятности.)
     - Функция подсчета очков по-прежнему точечная, что может быть неоптимальным.

## Эволюция LTR Model
Как мы оцениваем результат прогнозирования рейтинга?

Было предложено несколько показателей, которые обычно используются при оценке модели ранжирования:

- Бинарная релевантность
     - Средняя средняя точность (MAP)
     - Средний взаимный ранг (MRR)
- Оценка релевантности
     - Нормализованная дисконтированная совокупная прибыль (NDCG)
     - Ожидаемый взаимный ранг (ERR)
    
Как правило, бинарные меры учитывают только релевантные и сравнительные данные. нерелевантными, в то время как при градуированных показателях также будет учитываться ранжирование среди релевантных элементов. В этом случае при оценке рейтингового списка имеет значение степень релевантности.

## Labeling Issues
Вообще говоря, есть два подхода к маркировке набора данных ранжирования:

- Человеческое суждение
- Вывод из log поведения пользователя

Для первого подхода требуется огромная рабочая сила, чтобы обозначить релевантность каждого элемента для данного запроса. В реальном мире многие наборы данных не могут быть помечены таким образом, поэтому мы полагаемся на второй подход, который косвенно определяет предпочтения пользователя среди различных элементов.

Обычно парное предпочтение может быть получено из взаимодействия пользователя с результатом запроса. Например, используйте данные о кликах, чтобы сделать вывод о релевантности веб-поиска. Вот почему попарный подход в LTR может привлечь гораздо больше внимания, чем точечный метод: из-за доступности данных.


## From Pairwise to Listwise, and More
Недавно исследователи из Google обобщили структуру LambdaMART, чтобы предоставить теоретические основы модели ранжирования всех трех типов функции потерь (точечной, попарной, списочной) и прямой оптимизации всех популярных показателей ранжирования (NDCG, MAP, ... ). Фреймворк называется LambdaLoss (2018).

Готовая к производству реализация такого фреймворка также находится в открытом доступе в виде модуля ранжирования в популярной библиотеке TensorFlow. Также предлагается функция групповой оценки, которая может быть реализована в библиотеке.

Причина, по которой мы решили специально разработать вышеупомянутые модели, заключается в том, что они являются самой основой литературы LTR, цитируемой более тысячи раз.

И причина, по которой выбираются библиотеки, в основном та же: это самые современные популярные фреймворки с открытым исходным кодом в этой области.



## RankNet
Помните, что мы моделируем разницу оценок между данной парой (i, j) как вероятность, основанную на сигмовидной функции:

$Pr(i \succ j) = P_{ij} \equiv \frac{1}{1 + exp^{-(s_i - s_j)}}$
где

si=f(q,di) 

это точечный результат, полученный нашим алгоритмом f (q, d), который в RankNet сформулирован как двухуровневая нейронная сеть, параметризованная набором wk. (Или даже подумайте проще, пусть f (q, di) = wxi как линейный алгоритм.)

Учитывая распределение вероятностей p, энтропия определяется как: p⋅log21p. Теперь пусть yij∈ {0,1} будет фактической меткой данной пары (i, j). Функция потерь в приведенной выше настройке будет перекрестной энтропией:

$loss = -\sum_{i \neq j}{y_{ij}log_2P_{ij} + (1-y_{ij})log_2(1-P_{ij})}$

Кросс-энтропия измеряет, насколько близки два распределения вероятностей друг к другу. Естественно, это хорошая целевая функция для модели машинного обучения, моделирующей вероятность оптимизации. Используя технологию обратного распространения, мы можем численно найти веса модели в f (q, d), которые минимизируют потерю кросс-энтропии.

Обратите внимание, что приведенная выше потеря носит очень общий характер: это просто ожидаемая логарифмическая потеря или сумма кросс-энтропии из каждого обучающего примера, используемая для измерения того, насколько хорошо распределение модели приближается к эмпирическому распределению данных обучения (которое, в свою очередь, служит приближением к неизвестному истинному распределению, генерирующему обучающие данные). Мы можем легко поменять местами нейронную сеть с другими учащимися, что приведет к множеству различных попарных моделей LTR.

## LambdaNet
От RankNet до LambdaNet были достигнуты два важных усовершенствования.

1.Ускорение обучения за счет факторизации расчета градиента

2) Оптимизация по метрике ранжирования

## Gradient Factorization
Во-первых, LambdaNet - это **математически улучшенная версия RankNet**. Улучшение основано на факторизации вычисления градиента потери кросс-энтропии в контексте попарного обновления.

Учитывая потерю точечной кросс-энтропии как L:

$L = y_{ij}log_2P_{ij} + (1-y_{ij})log_2(1-P_{ij})$
 
Градиент (производная 1-го порядка потерь относительно параметра модели wk) может быть записан как:

$\frac{\partial L}{\partial w_k} = \frac{\partial L}{\partial s_i} \frac{\partial s_i}{\partial w_k} + \frac{\partial L}{\partial s_j} \frac{\partial s_j}{\partial w_k}$

Проще говоря, влияние изменения параметра модели wk будет происходить через результирующие изменения в оценках модели, а затем изменения в потерях. Теперь перепишите градиент общих потерь для всех обучающих пар {i, j}, удовлетворяющих i≻j:

$\begin{align}
\frac{\partial L_T}{\partial w_k} 
&= \sum_{\{i, j\}} \bigg[ \frac{\partial L}{\partial s_i} \frac{\partial s_i}{\partial w_k} + \frac{\partial L}{\partial s_j} \frac{\partial s_j}{\partial w_k} \bigg] \\ 
&= \sum_i \frac{\partial s_i}{\partial w_k} \bigg( \sum_{\forall j \prec i} \frac{\partial L(s_i, s_j)}{\partial s_i} \bigg) + \sum_j \frac{\partial s_j}{\partial w_k} \bigg( \sum_{\forall i \succ j} \frac{\partial L(s_i, s_j)}{\partial s_j} \bigg)
\end{align}$
 
при том, что:
$\frac{\partial L(s_i, s_j)}{\partial s_i} = - \frac{\partial L(s_i, s_j)}{\partial s_j} = log_2e\big[(1 - y_{ij}) - \frac{1}{1 + e^{s_i - s_j}}\big]$,
и переиндексация второго члена, мы получаем:

$\begin{align}
\frac{\partial L_T}{\partial w_k} 
&= \sum_i \frac{\partial s_i}{\partial w_k} \bigg[ \sum_{\forall j \prec i} \frac{\partial L(s_i, s_j)}{\partial s_i} + \sum_{\forall j \prec i} \frac{\partial L(s_j, s_i)}{\partial s_i} \bigg] \\
&= \sum_i \frac{\partial s_i}{\partial w_k} \bigg[ \sum_{\forall j \prec i} \frac{\partial L(s_i, s_j)}{\partial s_i} - \sum_{\forall j \succ i} \frac{\partial L(s_j, s_i)}{\partial s_j} \bigg] \\
&= \sum_i \frac{\partial s_i}{\partial w_k} \lambda_i
\end{align}$

Интуиция за вышеуказанным градиентом:

Для каждого документа в данном запросе существует компонент градиента, который мы обозначили как лямбда, который вычисляется путем рассмотрения всех сравниваемых с ним документов высшего и более низкого качества. Относительно худший документ подтолкнет текущий документ вверх, а относительно лучший - вниз.

Смысл вышеупомянутой факторизации заключается в том, что в процессе обучения, вместо обновления каждой пары документов, мы можем выполнять обновление для каждого запроса. А поскольку лямбда намного дешевле вычислить, весь процесс обучения может значительно ускориться.

## Ranking Metric Optimization
Поскольку мы моделируем разницу оценок пары документов в запросе как показатель вероятности, модель оптимизирует попарную корректность ранжирования, что может не быть в конечном итоге желаемой целью.

Помните, что цель ранжирования действительно измеряется (в идеале) позиционно-зависимым градуированным показателем, таким как NDCG. Но в приведенной выше установке NDCG напрямую не связан с минимизацией кросс-энтропии. Прямым и простым решением является использование NDCG в качестве критерия ранней остановки и определение с помощью набора данных проверки.

LambdaRank предлагает еще одно решение. Исследователь обнаружил, что во время обновления градиента с использованием понятия лямбда для каждой пары вместо вычисления только лямбда мы можем скорректировать лямбда путем изменения NDCG для этой пары при условии, что позиции двух элементов поменялись местами друг с другом.

Лямбда данного документа:

$\begin{align}
\lambda_i 
&= \bigg[ \sum_{\forall j \prec i} \frac{\partial L(s_i, s_j)}{\partial s_i} - \sum_{\forall j \succ i} \frac{\partial L(s_j, s_i)}{\partial s_j} \bigg] \\
&= \bigg[ \sum_{\forall j \prec i} \lambda_{ij} - \sum_{\forall j \succ i} \lambda_{ij} \bigg]
\end{align}$

Предлагаемый метод состоит в том, чтобы настроить попарную лямбду λij так, чтобы:

$\lambda_{ij} \equiv \frac{\partial L(s_i, s_j)}{\partial s_i} \cdot |\Delta NDCG_{ij}|$
где  ΔNDCGij  это изменение в NDCG, когда позиции i и j меняются местами.

Исследователь обнаружил, что при такой корректировке, без теоретических доказательств, модель эмпирически оптимизирует NDCG и, следовательно, дает лучшие общие результаты.

## LambdaMART
LambdaMART - это просто LambdaNet, но она заменяет базовую модель нейронной сети **деревьями регрессии с градиентным усилением** (или, в более общем смысле, машинами с градиентным усилением, GBM). GBM доказал свою надежность и производительность при решении реальных проблем.

Модель выигрывает несколько реальных крупномасштабных соревнований LTR.


## LambdaLoss
В исходной структуре LambdaRank и LambdaMART не проводилось никаких теоретических работ, чтобы математически доказать, что метрика ранжирования оптимизируется после корректировки расчета лямбда. Вывод основан исключительно на эмпирических работах, то есть на наблюдении за результатами из различных наборов данных и моделирования с экспериментами.

Исследователи из Google недавно (2018 г.) опубликовали обобщенную структуру под названием LambdaLoss, которая служит расширением исходной модели ранжирования и содержит подробные теоретические обоснования, подтверждающие, что модель действительно оптимизирует метрику ранжирования.

## Implement LambdaMART using lightgbm

In [132]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,0.50,1,3,3,0,0,3,1.0,1.0,0.000000,...,62,11089534,2,116,64034,13,3,0,0,0.0
1,0.50,1,3,0,3,0,3,1.0,0.0,1.000000,...,54,11089534,2,124,64034,1,2,0,0,0.0
2,0.00,1,3,0,2,0,3,1.0,0.0,0.666667,...,45,3,1,124,3344,14,67,0,0,0.0
3,0.50,1,3,0,3,0,3,1.0,0.0,1.000000,...,56,11089534,13,123,63933,1,3,0,0,0.0
4,0.25,1,3,0,3,0,3,1.0,0.0,1.000000,...,64,5,7,256,49697,1,13,0,0,0.0


In [133]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,0.50,13,2,0,2,1,2,1.0,0.0,1.0,...,35,1,0,266,25070,28,7,0,0,0.0
1,0.25,13,2,0,0,0,2,1.0,0.0,0.0,...,17,93,0,153,12860,65,158,0,0,0.0
2,0.75,13,2,0,1,0,2,1.0,0.0,0.5,...,19,0,0,153,1131,112,141,0,0,0.0
3,0.25,13,2,0,2,1,2,1.0,0.0,1.0,...,50,81775,0,560,61224,1,14,0,0,0.0
4,0.00,13,1,0,0,0,1,0.5,0.0,0.0,...,24,0,0,57953,15600,15,12,0,0,0.0


In [145]:
train_df = pd.read_csv('msrank_train.csv', index_col=0)
test_df = pd.read_csv('msrank_test.csv', index_col=0)

In [146]:
#from catboost.datasets import msrank_10k
#train_df, test_df = msrank_10k()

X_train = train_df.drop(['0', '1'], axis=1).values
y_train = train_df['0'].values

qids_train = train_df.groupby("1")["1"].count().to_numpy()

X_test = test_df.drop(['0', '1'], axis=1).values
y_test = test_df['0'].values

qids_test = test_df.groupby("1")["1"].count().to_numpy()


In [147]:
import lightgbm as lgb

In [148]:
model = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_test, y_test)],
    eval_group=[qids_test],
    eval_at=10,
    verbose=10,
)

[10]	valid_0's ndcg@10: 0.369033


KeyboardInterrupt: 

## Listwise LTR на tensorflow

Мы будем использовать tensorflow вместе с tensorflow_ranking, чтобы продемонстрировать, как мы можем построить модель PoC LTR в пределах 200 строк кода Python.

Обратите внимание, что обычно использование tensorflow требует гораздо больше усилий, поскольку это низкоуровневый фреймворк для моделирования машинного обучения.

Мы рассматриваем задачу ранжирования над ANTIQUE, набором данных с ответами на вопросы. Учитывая запрос и список ответов, цель состоит в том, чтобы максимизировать метрику, связанную с рангом (NDCG).

**A Question Answering Dataset**

ANTIQUE - это общедоступный набор данных для неактуальных ответов на вопросы в открытом домене, собранный через Yahoo! ответы.

На каждый вопрос есть список ответов, актуальность которых оценивается по шкале от 1 до 5.

Размер списка может варьироваться в зависимости от запроса, поэтому мы используем фиксированный «размер списка» 50, где список либо усечен, либо дополнен фиктивными значениями.

Этот набор данных подходит для сценария обучения ранжированию. Набор данных разделен на 2206 запросов для обучения и 200 запросов для тестирования.

Загрузим файл обучения, тестовых данных и словарь.

In [141]:
#!wget -O "vocab.txt" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/vocab.txt"
#!wget -O "train.tfrecords" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/ELWC/train.tfrecords"
#!wget -O "test.tfrecords" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking//ELWC/test.tfrecords"

## Data Formats for Ranking

Для представления данных ранжирования протобуферы представляют собой расширяемые структуры, подходящие для хранения данных в сериализованном формате либо локально, либо распределенно.

https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.10/tensorflow/g3doc/how_tos/tool_developers/index.md

Ранжирование обычно состоит из характеристик, соответствующих каждому из сортируемых примеров. Кроме того, для ранжирования полезны признаки, связанные с запросом, пользователем или сессии. Мы называем их признаками контекста, поскольку они не зависят от примеров.

Мы используем популярный протокол tf.Example для представления признаков для контекста и каждого из примеров. Мы используем протобуфер ExampleListWithContext (ELWC) для хранения контекста как tf.Example и списка примеров, которые будут ранжироваться как список протоколов tf.Example.

Здесь определяется протбуфер ExampleListWithContext.

Создадим фиктивные данные в формате ELWC. Мы будем использовать эти фиктивные данные, чтобы показать, как выглядит прототип.


Загрузите и установите пакеты TensorFlow Ranking и TensorFlow Serving.

In [149]:
import tensorflow as tf
import tensorflow_ranking as tfr
from tensorflow_serving.apis import input_pb2

2021-11-12 20:18:03.645909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 20:18:03.646009: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.4.0 and strictly below 2.7.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.7.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's 

In [150]:
from google.protobuf import text_format

CONTEXT = text_format.Parse(
    """
    features {
      feature {
        key: "query_tokens"
        value { bytes_list { value: ["this", "is", "a", "relevant", "question"] } }
      }
    }""", tf.train.Example())

In [152]:
EXAMPLES = [
    text_format.Parse(
    """
    features {
      feature {
        key: "document_tokens"
        value { bytes_list { value: ["this", "is", "a", "relevant", "answer"] } }
      }
      feature {
        key: "relevance"
        value { int64_list { value: 5 } }
      }
    }""", tf.train.Example()),
    text_format.Parse(
        """
    features {
      feature {
        key: "document_tokens"
        value { bytes_list { value: ["irrelevant", "data"] } }
      }
      feature {
        key: "relevance"
        value { int64_list { value: 1 } }
      }
    }""", tf.train.Example()),
]

In [153]:
ELWC = input_pb2.ExampleListWithContext()
ELWC.context.CopyFrom(CONTEXT)
for example in EXAMPLES:
    example_features = ELWC.examples.add()
    example_features.CopyFrom(example)

In [154]:
print(ELWC)

examples {
  features {
    feature {
      key: "document_tokens"
      value {
        bytes_list {
          value: "this"
          value: "is"
          value: "a"
          value: "relevant"
          value: "answer"
        }
      }
    }
    feature {
      key: "relevance"
      value {
        int64_list {
          value: 5
        }
      }
    }
  }
}
examples {
  features {
    feature {
      key: "document_tokens"
      value {
        bytes_list {
          value: "irrelevant"
          value: "data"
        }
      }
    }
    feature {
      key: "relevance"
      value {
        int64_list {
          value: 1
        }
      }
    }
  }
}
context {
  features {
    feature {
      key: "query_tokens"
      value {
        bytes_list {
          value: "this"
          value: "is"
          value: "a"
          value: "relevant"
          value: "question"
        }
      }
    }
  }
}




**Dependencies and Global Variables**

Здесь мы определяем пути обучения и тестирования, а также гиперпараметры модели.

In [155]:
# Store the paths to files containing training and test instances.
_TRAIN_DATA_PATH = "train.tfrecords"
_TEST_DATA_PATH = "test.tfrecords"

# Store the vocabulary path for query and document tokens.
_VOCAB_PATH = "vocab.txt"

# The maximum number of documents per query in the dataset.
# Document lists are padded or truncated to this size.
_LIST_SIZE = 50

# The document relevance label.
_LABEL_FEATURE = "relevance"

# Padding labels are set negative so that the corresponding examples can be
# ignored in loss and metrics.
_PADDING_LABEL = -1

# Learning rate for optimizer.
_LEARNING_RATE = 0.05

# Parameters to the scoring function.
_BATCH_SIZE = 32
_HIDDEN_LAYER_DIMS = ["64", "32", "16"]
_DROPOUT_RATE = 0.8
_GROUP_SIZE = 1  # Pointwise scoring.

# Location of model directory and number of training steps.
_MODEL_DIR = "ranking_model_dir"
_NUM_TRAIN_STEPS = 15 * 1000

**Components of a Ranking Estimator**

Общие компоненты рейтингового оценщика показаны ниже.

Ключевые компоненты библиотеки:

- Input Reader
- Tranform Function
- Scoring Function
- Ranking Losses
- Ranking Metrics
- Ranking Head
- Model Builder


Столбцы фичей - это абстракции TensorFlow, которые используются для сбора обширной информации о каждой фичи. Это позволяет легко преобразовывать широкий спектр необработанных фичей и взаимодействовать с ними в одном обьекте.

В соответствии с нашими форматами ввода для ранжирования, такими как формат ELWC, мы создаем столбцы фичей для всех признаков

In [156]:
_EMBEDDING_DIMENSION = 20


def context_feature_columns():
    """Returns context feature names to column definitions."""
    sparse_column = tf.feature_column.categorical_column_with_vocabulary_file(
    key="query_tokens",
    vocabulary_file=_VOCAB_PATH)
    query_embedding_column = tf.feature_column.embedding_column(
    sparse_column, _EMBEDDING_DIMENSION)
    return {"query_tokens": query_embedding_column}


def example_feature_columns():
    """Returns the example feature columns."""
    sparse_column = tf.feature_column.categorical_column_with_vocabulary_file(
    key="document_tokens",
    vocabulary_file=_VOCAB_PATH)
    document_embedding_column = tf.feature_column.embedding_column(
    sparse_column, _EMBEDDING_DIMENSION)
    return {"document_tokens": document_embedding_column}

### Reading Input Data using input_fn

Устройство чтения ввода считывает данные из постоянного хранилища для создания необработанных плотных и разреженных тензоров соответствующего типа для каждой фичи. Примеры признаков представлены трехмерными тензорами (где измерения соответствуют запросам, примерам и значениям признака). Особенности контекста представлены двумерными тензорами (где измерения соответствуют запросам и значениям признаков).

https://www.tensorflow.org/tutorials/structured_data/feature_columns

In [157]:
def input_fn(path, num_epochs=None):
    context_feature_spec = tf.feature_column.make_parse_example_spec(
        context_feature_columns().values())
    label_column = tf.feature_column.numeric_column(
        _LABEL_FEATURE, dtype=tf.int64, default_value=_PADDING_LABEL)
    example_feature_spec = tf.feature_column.make_parse_example_spec(
        list(example_feature_columns().values()) + [label_column])
    dataset = tfr.data.build_ranking_dataset(
        file_pattern=path,
        data_format=tfr.data.ELWC,
        batch_size=_BATCH_SIZE,
        list_size=_LIST_SIZE,
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        reader=tf.data.TFRecordDataset,
        shuffle=False,
        num_epochs=num_epochs)
    features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    label = tf.squeeze(features.pop(_LABEL_FEATURE), axis=2)
    label = tf.cast(label, tf.float32)

    return features, label

**Feature Transformations with transform_fn**

Функция преобразования принимает необработанные dense or sparse features из устройства чтения ввода, применяет подходящие преобразования для возврата плотных представлений для каждой фичи. Это важно перед передачей этих признаков в нейронную сеть, поскольку слои нейронных сетей обычно принимают dense features в качестве входных данных.

Функция преобразования обрабатывает любые преобразования признаков, определенные пользователем. Для работы с разреженными признаками, такими как текстовые данные, мы предоставляем удобную возможность для создания общих встраиваний на основе столбцов фичей.

In [158]:
def make_transform_fn():
    def _transform_fn(features, mode):
        """Defines transform_fn."""
        context_features, example_features = tfr.feature.encode_listwise_features(
        features=features,
        context_feature_columns=context_feature_columns(),
        example_feature_columns=example_feature_columns(),
        mode=mode,
        scope="transform_layer")

        return context_features, example_features
    return _transform_fn

**Feature Interactions using scoring_fn**
Затем мы переходим к функции оценки, которая, возможно, лежит в основе модели рейтинга TF. Идея состоит в том, чтобы вычислить оценку релевантности для (набора) пар запрос-документ. Модель TF-Ranking будет использовать данные обучения для изучения этой функции.

Здесь мы формулируем функцию оценки с использованием сети прямого распространения. Функция использует характеристики одного примера (например, пары запрос-документ) и выдает оценку релевантности.

In [159]:
def make_score_fn():
    """Returns a scoring function to build `EstimatorSpec`."""

    def _score_fn(context_features, group_features, mode, params, config):
        """Defines the network to score a group of documents."""
        with tf.compat.v1.name_scope("input_layer"):
            context_input = [
              tf.compat.v1.layers.flatten(context_features[name])
            for name in sorted(context_feature_columns())
            ]
            group_input = [
              tf.compat.v1.layers.flatten(group_features[name])
              for name in sorted(example_feature_columns())
            ]
            input_layer = tf.concat(context_input + group_input, 1)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)
        cur_layer = input_layer
        cur_layer = tf.compat.v1.layers.batch_normalization(
        cur_layer,
        training=is_training,
        momentum=0.99)

        for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
            cur_layer = tf.compat.v1.layers.dense(cur_layer, units=layer_width)
            cur_layer = tf.compat.v1.layers.batch_normalization(
            cur_layer,
            training=is_training,
            momentum=0.99)
        cur_layer = tf.nn.relu(cur_layer)
        cur_layer = tf.compat.v1.layers.dropout(
          inputs=cur_layer, rate=_DROPOUT_RATE, training=is_training)
        logits = tf.compat.v1.layers.dense(cur_layer, units=_GROUP_SIZE)
        return logits

    return _score_fn

## Losses, Metrics and Ranking Head

**Evaluation Metrics**
 

In [160]:
def eval_metric_fns():
    """Returns a dict from name to metric functions.

  This can be customized as follows. Care must be taken when handling padded
  lists.

  def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
  metric_fns["auc"] = _auc

  Returns:
    A dict mapping from metric name to a metric function with above signature.
  """
    metric_fns = {}
    metric_fns.update({
      f"metric/ndcg@{topn}": tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns


### Ranking Losses

In [161]:
# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.

_LOSS = tfr.losses.RankingLossKey.APPROX_NDCG_LOSS
loss_fn = tfr.losses.make_loss_fn(_LOSS)

### Ranking Head

В рабочем процессе оценщика Head - это абстракция, которая инкапсулирует потери и соответствующие показатели. Head легко взаимодействует с оценщиком, что требует от пользователя определения функции оценки и определения потерь и расчета показателей.

In [162]:
optimizer = tf.compat.v1.train.AdagradOptimizer(
learning_rate=_LEARNING_RATE)


def _train_op_fn(loss):
    """Defines train op used in ranking head."""
    update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
    minimize_op = optimizer.minimize(
      loss=loss, global_step=tf.compat.v1.train.get_global_step())
    train_op = tf.group([update_ops, minimize_op])
    return train_op


ranking_head = tfr.head.create_ranking_head(
      loss_fn=loss_fn,
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

### Putting It All Together in a Model Builder

Теперь мы готовы собрать все вышеперечисленные компоненты вместе и создать одну сущность модели, который можно использовать для обучения и оценки модели.

In [163]:
model_fn = tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          transform_fn=make_transform_fn(),
          group_size=_GROUP_SIZE,
          ranking_head=ranking_head)

INFO:tensorflow:Building groupwise ranking model.


### Train and evaluate the ranker

In [164]:
def train_and_eval_fn():
    """Train and eval function used by `tf.estimator.train_and_evaluate`."""
    run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=1000)
    ranker = tf.estimator.Estimator(
      model_fn=model_fn,
      model_dir=_MODEL_DIR,
      config=run_config)

    train_input_fn = lambda: input_fn(_TRAIN_DATA_PATH)
    eval_input_fn = lambda: input_fn(_TEST_DATA_PATH, num_epochs=1)

    train_spec = tf.estimator.TrainSpec(
      input_fn=train_input_fn, max_steps=_NUM_TRAIN_STEPS)
    eval_spec = tf.estimator.EvalSpec(
      name="eval",
      input_fn=eval_input_fn,
      throttle_secs=15)
    return (ranker, train_spec, eval_spec)

In [165]:
ranker, train_spec, eval_spec = train_and_eval_fn()
tf.estimator.train_and_evaluate(ranker, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': 'ranking_model_dir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distri

(None, None)

### Launch TensorBoard

In [158]:
%load_ext tensorboard
%tensorboard --logdir="ranking_model_dir" --port 12345

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

### Generating Predictions

Мы показываем, как создавать прогнозы по функциям набора данных. Мы предполагаем, что метка отсутствует и должна быть выведена с использованием модели ранжирования.

Подобно input_fn, используемому для обучения и оценки, pred_input_fn считывает данные в формате ELWC и сохраняет их как TFRecords для создания функций. Мы устанавливаем количество эпох равным 1, чтобы генератор прекратил итерацию, когда достигнет конца набора данных. Кроме того, при чтении точки данных не перемешиваются, поэтому поведение функции predic () является детерминированным.

In [167]:
def predict_input_fn(path):
    context_feature_spec = tf.feature_column.make_parse_example_spec(
        context_feature_columns().values())
    example_feature_spec = tf.feature_column.make_parse_example_spec(
        list(example_feature_columns().values()))
    dataset = tfr.data.build_ranking_dataset(
        file_pattern=path,
        data_format=tfr.data.ELWC,
        batch_size=_BATCH_SIZE,
        list_size=_LIST_SIZE,
        context_feature_spec=context_feature_spec,
        example_feature_spec=example_feature_spec,
        reader=tf.data.TFRecordDataset,
        shuffle=False,
        num_epochs=1)
    features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    return features

Будем генерировать прогнозы на тестовом наборе данных, где мы рассматриваем только контекст и примеры функций и прогнозируем метки. Pred_input_fn генерирует прогнозы для пакета точек данных. Пакетная обработка позволяет нам перебирать большие наборы данных, которые невозможно загрузить в память.

In [168]:
predictions = ranker.predict(input_fn=lambda: predict_input_fn("test.tfrecords"))

ranker.predict возвращает генератор, который мы можем перебирать для создания прогнозов, пока генератор не будет исчерпан.

In [169]:
x = next(predictions)
assert len(x) == _LIST_SIZE  # Note that this includes padding.

INFO:tensorflow:vocabulary_size = 30522 in query_tokens is inferred from the number of elements in the vocabulary_file vocab.txt.
INFO:tensorflow:vocabulary_size = 30522 in document_tokens is inferred from the number of elements in the vocabulary_file vocab.txt.


2021-11-12 20:27:05.360829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 20:27:05.360953: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-11-12 20:27:05.361015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-11-12 20:27:05.364624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-11-12 20:27:05.364703: W tensorflow/stream_executor/platform/default/dso_loader

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:vocabulary_size = 30522 in query_tokens is inferred from the number of elements in the vocabulary_file vocab.txt.
INFO:tensorflow:vocabulary_size = 30522 in document_tokens is inferred from the number of elements in the vocabulary_file vocab.txt.
INFO:tensorflow:vocabulary_size = 30522 in query_tokens is inferred from the number of elements in the vocabulary_file vocab.txt.
INFO:tensorflow:vocabulary_size = 30522 in document_tokens is inferred from the number of elements in the vocabulary_file vocab.txt.


/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  if __name__ == '__main__':
/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/keras/legacy_tf_layers/core.py:523: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  del sys.path[0]
/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. 

INFO:tensorflow:Done calling model_fn.


/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/keras/legacy_tf_layers/core.py:401: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/data/home/irsafilo/course/shad/venv/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.


INFO:tensorflow:Graph was finalized.


2021-11-12 20:27:06.932023: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Restoring parameters from ranking_model_dir/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2021-11-12 20:27:07.297873: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
